<h1>Books assignment</h1>

In [1]:
import numpy as np 
import pandas as pd 
import os

In [58]:
path_ratings="~/ukol/BX-Book-Ratings.csv"
df_ratings=pd.read_csv(path_ratings, on_bad_lines="warn", delimiter=";", encoding = "ISO-8859-1")
df_ratings.dropna()
df_ratings['ISBN'] = df_ratings['ISBN'].str.replace('[^\d]', '', regex=True)  # ISBN mess
df_ratings.head(5)

,User-ID,ISBN,Book-Rating
0,276725,034545104,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615,3
4,276729,0521795028,6


In [166]:
#df_ratings.describe()
#df_ratings.shape
#df_ratings.info()
#df_ratings.dtypes

In [165]:
path_books="~/ukol/BX-Books.csv"
df_books=pd.read_csv(path_books, on_bad_lines="skip", delimiter=";", encoding = "ISO-8859-1", dtype={"ISBN": "string", "Year-Of-Publication": "string"})
df_books.dropna()
df_books['ISBN'] = df_books['ISBN'].str.replace('[^\d]', '', regex=True)
df_books = df_books.drop_duplicates(subset='ISBN')
df_books_shorter = df_books.drop(['Publisher','Image-URL-S','Image-URL-M','Image-URL-L','Year-Of-Publication'],axis=1)
df_books_shorter.head(5)

,ISBN,Book-Title,Book-Author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [92]:
def full_name(isbn):
    book_name = df_books_shorter[df_books_shorter['ISBN']==isbn]['Book-Title']
    book_author = df_books_shorter[df_books_shorter['ISBN']==isbn]['Book-Author']
    if ((book_name.empty) or (book_author.empty)):
        return "", ""
    else:
        return book_name.iloc[0], book_author.iloc[0]

In [96]:
# improvised test
bn, ba = full_name("0002005018")
# should return Clara Callan, Richard Bruce Wright
#bn, ba = full_name("bleble")
# should return empty strings
if (bn and ba):
    print("You should read {} written by {}.".format(bn, ba))



You should read Clara Callan written by Richard Bruce Wright.


In [85]:
df_ratings_mensi = df_ratings.head(50000)
df_ratings_mensi = df_ratings_mensi.tail(10000)

In [147]:
def very_dummy_recommendation(isbn):
    min_rating = 7
    cetli_to = df_ratings[df_ratings['ISBN']==isbn]
    maji_radi = cetli_to.loc[(cetli_to['Book-Rating'] >= min_rating)]
    if maji_radi.empty:
        print("Insufficient data for a reliable recommendation. Your taste is unique.")
    else:
        maji_jeste_radi = maji_radi  # hack to preserve the format        
        for index, row in df_ratings.iterrows():
            for i in range(maji_radi.shape[0]):                
                if (row['User-ID'] == maji_radi['User-ID'].iloc[i]):
                    #print ("yes, {}, {}".format(row['User-ID'],maji_radi['User-ID'].iloc[i]))                  
                    row_df = pd.DataFrame([row])
                    maji_jeste_radi = pd.concat([maji_jeste_radi, row_df], axis=0, ignore_index=True)
        maji_jeste_radi = maji_jeste_radi[maji_jeste_radi['ISBN']!=isbn] #drop the original book from the list
        maji_jeste_radi = maji_jeste_radi.drop(['User-ID'],axis=1)       # to ease the grouping
        ma_jich_rado_vic = maji_jeste_radi[maji_jeste_radi.groupby('ISBN')['ISBN'].transform('count').ge(2)]
                            # to keep only books rated by more than one relevant user
        prumery = ma_jich_rado_vic.groupby('ISBN')['Book-Rating'].agg(['mean', 'count']).sort_values(by='mean',ascending=False)
        
    return prumery

In [151]:
favourite_isbn = "0345339703"  #"The Fellowship of the Ring (The Lord of the Rings, Part 1)"
                               # loooong computing!
poradi = very_dummy_recommendation(favourite_isbn)
poradi


,mean,count
ISBN,,
0764502611,10.0,2
043936213,10.0,3
0345295684,10.0,2
0762409533,10.0,2
0671705091,10.0,2
...,...,...
0553580051,0.0,2
0425116840,0.0,2
0553580221,0.0,2


In [156]:
max_recommendations = 20  # or 10, or...
reasonable_rating = 7
i = 0
for index, row in poradi.iterrows():
    if ((row[0] > reasonable_rating) and (i < max_recommendations)):
        bname, bauthor = full_name(index)
        if (bname and bauthor):            
            print("You should read {} written by {} (avg. rating {}).".format(bname, bauthor, row[0]))
            i+=1


You should read Windows 98 for Dummies written by Andy  Rathbone (avg. rating 10.0).
You should read Harry Potter and the Sorcerer's Stone (Book 1) written by J. K. Rowling (avg. rating 10.0).
You should read Dragonflight written by Anne Mccaffrey (avg. rating 10.0).
You should read TOLKIEN MAGNETIC POSTCARDS(tm) 12 Full-color Magnetic Postcards to Send or Save written by Brothers Hildebrandt (avg. rating 10.0).
You should read A Knight in Shining Armor written by Jude Deveraux (avg. rating 10.0).
You should read The Official Scrabble Players Dictionary (Third Edition) written by Merriam-Webster (avg. rating 10.0).
You should read On Writing written by Stephen King (avg. rating 10.0).
You should read The Lion, the Witch and the Wardrobe (The Chronicles of Narnia) written by C. S. Lewis (avg. rating 10.0).
You should read Doomsday Book written by Connie Willis (avg. rating 10.0).
You should read Conceived Without Sin written by Bud Macfarlane Jr. (avg. rating 10.0).
You should read New 

In [164]:
path_ratings="~/ukol/recommendations_" + favourite_isbn
poradi.to_csv(path_ratings)